## 관련 패키지 import

In [1]:
import hashlib # hash 함수용 sha256" 사용할 라이브러리
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify

## Blockchain 객체 생성

In [2]:
class Blockchain(object):
    
    def __init__(self):
        self.chain = []                                   # chain에 여러 block들 들어옴
        self.current_transaction = []                     # 임시 transaction 넣어줌
        self.nodes = set()                                # Node 목록을 보관
        self.new_block(previous_hash=1, proof=100)        # genesis block 생성

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode() 
        return hashlib.sha256(block_string).hexdigest()   # hash 라이브러리로 sha256 사용
    
    @property
    def last_block(self):
        return self.chain[-1]                             # 체인의 마지막 블록 가져오기!!

    
    @staticmethod
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()          # 전 proof와 구할 proof 문자열 연결
        guess_hash = hashlib.sha256(guess).hexdigest()    # 이 hash 값 저장
        return guess_hash[:4] == "0000"                  # 앞 4자리가 0000 이면 True (알맞은 nonce값을 찾음)

    def pow(self, last_proof):
        proof = random.randint(-1000000,1000000)
        while self.valid_proof(last_proof, proof) is False: # valid proof 함수 활용(아래 나옴), 맞을 때까지 반복적으로 검증
            proof = random.randint(-1000000,1000000)
        return proof
    
    ## 새로운 거래 생성
    def new_transaction(self, sender, recipient, amount, otherUniqueIdentifiers=None):
    # otherUniqueIdentifiers가 딕셔너리이고, 'hash' 키가 존재하는지 확인
        unique_hash = otherUniqueIdentifiers.get('hash') if otherUniqueIdentifiers and 'hash' in otherUniqueIdentifiers else None

        # 들여쓰기 수정
        self.current_transaction.append(
            {
                'sender': sender,
                'recipient': recipient,
                'amount': amount,
                'timestamp': time(),
                'sbom-hash': unique_hash,  # hash 값만 저장
            }
        )
        return self.last_block['index'] + 1



    
    ## 새로운 블록 생성
    def new_block(self, proof, previous_hash=None):
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(), # timestamp from 1970
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)     
        return block

    def valid_chain(self, chain):
        last_block = chain[0] 
        current_index = 1

        while current_index < len(chain): 
            block = chain[current_index]
            print('%s' % last_block)
            print('%s' % block)
            print("\n--------\n")
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
        return True


## Blockchain 객채를 기반으로 노드 생성

In [3]:
blockchain = Blockchain()
my_ip = '0.0.0.0'
my_port = '5000'
node_identifier = 'node_'+my_port
mine_owner = 'master'
mine_profit = 0.1

In [ ]:
app = Flask(__name__)

# 전체 블록체인의 정보를 조회
@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain, 
        'length' : len(blockchain.chain), 
    }
    return jsonify(response), 200

# 새로운 거래를 블록체인에 추가
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json() 
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount'] 

    if not all(k in values for k in required):
        return 'missing values', 400

    index = blockchain.new_transaction(values['sender'],values['recipient'],values['amount'])
        
    response = {'message' : 'Transaction will be added to Block {%s}' % index}
    return jsonify(response), 201

# SBOM 거래를 블록체인에 추가
@app.route('/sbom/new', methods=['POST'])
def new_sbom_transaction():
    values = request.get_json()
    
    # 필수 값 검증
    required = ['sender', 'recipient', 'amount', 'otherUniqueIdentifiers']
    if not all(k in values for k in required):
        return 'Missing values', 400

    # new_transaction 메소드 호출, otherUniqueIdentifiers 딕셔너리 전체를 인자로 전달
    index = blockchain.new_transaction(
        sender=values['sender'],
        recipient=values['recipient'],
        amount=values['amount'],
        otherUniqueIdentifiers=values['otherUniqueIdentifiers']
    )
    
    response = {'message': 'Transaction will be added to Block {}'.format(index)}
    return jsonify(response), 201


# 새로운 블록을 채굴
@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")    
    last_block = blockchain.last_block
    last_proof = last_block['nonce']
    proof = blockchain.pow(last_proof)  

    blockchain.new_transaction(
        sender=mine_owner, 
        recipient=node_identifier, 
        amount=mine_profit # coinbase transaction 
    )
 
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")

    response = {
        'message' : 'new block found',
        'index' : block['index'],
        'transactions' : block['transactions'],
        'nonce' : block['nonce'],
        'previous_hash' : block['previous_hash']
    }
          
    return jsonify(response), 200


if __name__ == '__main__':
    app.run(host=my_ip, port=my_port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.17.71.92:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Mar/2024 17:25:11] "POST /sbom/new HTTP/1.1" 201 -


MINING STARTED


127.0.0.1 - - [05/Mar/2024 17:25:16] "GET /mine HTTP/1.1" 200 -


MINING FINISHED


127.0.0.1 - - [05/Mar/2024 17:25:19] "GET /chain HTTP/1.1" 200 -


chain info requested!!
